In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.linear_model import LinearRegression
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# Load and Preprocess Data
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df.fillna(method='ffill', inplace=True)  # Forward fill missing values
    df.fillna(method='bfill', inplace=True)  # Backward fill as extra safety
    df["Year"] = df["Year"].astype(int)
    return df

#  Get Inflation Data for Selected Years
def get_inflation_data(df, start_year=None, end_year=None):
    if start_year and end_year:
        filtered_df = df[(df["Year"] >= start_year) & (df["Year"] <= end_year)]
        if filtered_df.empty:
            print("⚠ No data found for the selected years!")
        return filtered_df
    return df

# Train Inflation Model (Fixing NaN Issue)
def train_inflation_model(df):
    X = df[["Year"]]
    y = df.drop(columns=["Year"])

    # 🛠 Fix: Fill NaN values to avoid model errors
    y.fillna(y.mean(), inplace=True)

    models = {}
    for col in y.columns:
        model = LinearRegression()
        model.fit(X, y[col])  # No NaN issue now!
        models[col] = model
    return models

#  Predict Future Inflation
def predict_inflation(models, start_year, end_year):
    future_years = np.arange(start_year, end_year + 1).reshape(-1, 1)
    predictions = {col: models[col].predict(future_years) for col in models.keys()}

    pred_df = pd.DataFrame(predictions)
    pred_df.insert(0, "Year", future_years.flatten())
    file_name = f"Inflation_Prediction_{start_year}to{end_year}.csv"
    pred_df.to_csv(file_name, index=False)

    return pred_df, file_name, future_years.flatten().tolist()  # Convert array to list for PDF

#  Enhanced Visualization
def visualize_inflation(df, start_year, end_year, save_path="inflation_trends.png"):
    df_filtered = df[(df["Year"] >= start_year) & (df["Year"] <= end_year)]

    if df_filtered.empty:
        print("⚠ No data available for visualization!")
        return None

    #  Stylish Seaborn Graph
    plt.figure(figsize=(12, 6))
    sns.set_theme(style="whitegrid")

    for col in df_filtered.columns[1:]:
        plt.plot(df_filtered["Year"], df_filtered[col], label=col, marker='o', markersize=4)

    plt.xlabel("Year", fontsize=12)
    plt.ylabel("Inflation Rate", fontsize=12)
    plt.title("Inflation Trends Over Selected Years", fontsize=14, fontweight="bold", color="darkblue")
    plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
    plt.grid(alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()

    return save_path  # Return saved file path for PDF inclusion

#  Generate PDF Report (Ensuring Everything is Captured)
def generate_pdf_report(future_years, predictions, visualization_path, start_year, end_year):
    report_path = f"Inflation_Report_{start_year}to{end_year}.pdf"
    doc = SimpleDocTemplate(report_path, pagesize=letter)
    elements = []

    # *Title Styling*
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        "TitleStyle",
        parent=styles["Title"],
        fontSize=18,
        textColor=colors.darkblue,
        spaceAfter=10,
        alignment=1
    )

    # *Add Title*
    elements.append(Paragraph(f"Inflation Report ({start_year}-{end_year})", title_style))
    elements.append(Spacer(1, 12))

    # *Create Table*
    col_headers = ["Year"] + [col[:10] + "..." if len(col) > 12 else col for col in predictions.keys()]
    data = [col_headers]

    for i, year in enumerate(future_years):
        row = [year] + [f"{predictions[col][i]:.2f}" for col in predictions]
        data.append(row)

    table = Table(data)
    table.setStyle(TableStyle([
        ("BACKGROUND", (0, 0), (-1, 0), colors.darkblue),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
        ("ALIGN", (0, 0), (-1, -1), "CENTER"),
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("FONTSIZE", (0, 1), (-1, -1), 9),
        ("BACKGROUND", (0, 1), (-1, -1), colors.lightgrey),
        ("GRID", (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(table)
    elements.append(Spacer(1, 20))

    # *Insert Visualization Below*
    if visualization_path:
        img = Image(visualization_path, width=400, height=300)
        elements.append(img)
    else:
        elements.append(Paragraph("⚠ No graph available due to missing data.", styles["Normal"]))

    # *Build PDF*
    doc.build(elements)
    print(f"PDF report saved: {report_path}")
    return report_path

# Load Dataset
file_path = r"D:\Finsight-Bharat\CPI_Inflation_India_1970_2024.csv"
df = load_and_preprocess_data(file_path)

# User Inputs
view_start = int(input(" Enter start year to view inflation data: "))
view_end = int(input(" Enter end year to view inflation data: "))
print(get_inflation_data(df, view_start, view_end))

viz_start = int(input("Enter start year for visualization: "))
viz_end = int(input("Enter end year for visualization: "))
visualization_path = visualize_inflation(df, viz_start, viz_end)

pred_start = int(input("Enter start year for prediction: "))
pred_end = int(input("Enter end year for prediction: "))
models = train_inflation_model(df)
pred_df, report_file, future_years = predict_inflation(models, pred_start, pred_end)

#  Generate Stylish Report
pdf_report = generate_pdf_report(future_years, pred_df, visualization_path, pred_start, pred_end)

print(" Visualization saved at:", visualization_path)
print(" Download report:", pdf_report)

C:\Users\PRAKASH\AppData\Local\Temp\ipykernel_7616\1582571732.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)  # Forward fill missing values
C:\Users\PRAKASH\AppData\Local\Temp\ipykernel_7616\1582571732.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)  # Backward fill as extra safety


    Year  CPI (Index)  Inflation Rate (%)
43  2013       698.74                8.06
44  2014       763.64                9.29
45  2015       798.29                4.54
46  2016       840.53                5.29
47  2017       906.69                7.87
48  2018       948.11                4.57
49  2019      1024.52                8.06
⚠ No data available for visualization!
PDF report saved: Inflation_Report_2026to2035.pdf
 Visualization saved at: None
 Download report: Inflation_Report_2026to2035.pdf


C:\Users\PRAKASH\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\PRAKASH\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
